<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/20_Sen_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.datasets import load_breast_cancer
import warnings
warnings.filterwarnings('ignore')